In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [2]:
#!pip install robobrowser

In [3]:
# !pip install pickledb pymongo elasticsearch==6.8 oauth2client gspread df2gspread \
# robobrowser beautifulsoup4 lxml yourls Werkzeug==0.16.1 paramiko xlsxwriter pyminizip \
# html5lib zodiac_sign python-pptx plotly df2gspread gspread

In [4]:
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)    
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)  # сюда json файл если он не в той же папке
            creds = flow.run_local_server(port=0)        
            
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)    
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
    
def to_float(x):
    try:
        return(float(x.replace("\xa0", '').replace(" ", '').strip()))
    except Exception as e:
        return np.nan

In [5]:
def cre_get(es, since):
    
    chunk = 10000
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", 
                                       ascending=False).drop_duplicates('userId', keep='first').reset_index(drop=True)
    df.index = df['userId']
      
            
    return df['score'].to_dict()

In [6]:
def mm_requestId(x):
    if type(x)==dict and 'NOTE' in x.keys():
        if x['NOTE'].find('HTTP_RESPONSE_BODY') >0:
            htr = json.loads(x['NOTE'][x['NOTE'].find('HTTP_RESPONSE_BODY')+20:])
            htr = htr['requestId'] if 'requestId' in htr.keys() else np.nan
#             print(htr)
        else: 
            htr = np.nan
    else: 
        htr = np.nan
    return htr

In [7]:
def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
#            "product._id": {"$in": [ObjectId("5e9846be79dd27000141f476"),
# ObjectId("5ef4473acf22a90001010445")]},
           #"product._id": {"$in": [ObjectId(product) for product in products]}}
            "status": {"$nin": [ "ERROR", "NEW1"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1, "clickId": 1,
                                                                "whence":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['productId'] = df['product'].apply(lambda x: str(x['_id']))
    df['product_type'] = df['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else np.nan)
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])
    df['phone'] = df['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)

    return df


def get_kibana_claims(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM"}},
#                          {"match_phrase":{"params.sourceGlobal.keyword":{"query": sourceGlobal }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
                  'transaction_id', 'clickId', 'sourceGlobal', 'utm_content','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [8]:
def get_kibana_postback(es, since):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"POSTBACK"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                'transaction_id', 'externalId','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [9]:
since = date(2021,1,4)
till = date(2021, 12, 30)

In [10]:
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')

dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)

dr.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claimReport.pickle.gz')
dc.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claims.pickle.gz')

def AR_count(score, productType, status):
    koef = 1
    check = [2200, 1700, 1100, 5500]
    ARlist = [[0.00499500499500499, 0.00305343511450382, 0.0714285714285714, 0.0], 
              [0.00380818053596615, 0.00871508379888268, 0.0768345323741007, 0.0], 
              [0.00685378590078329, 0.0136418349605457, 0.0846340617412418, 0.0053475935828877], 
              [0.0111028867505552, 0.0255610972568579, 0.0980751604032997, 0.0116279069767442], 
              [0.00938967136150235, 0.0349985043374215, 0.126042632066728, 0.0], 
              [0.0109689213893967, 0.0407338308457712, 0.140204271123491, 0.0], 
              [0.0141884222474461, 0.0514050719671008, 0.113486842105263, 0.0188679245283019], 
              [0.0150753768844221, 0.0687361419068736, 0.0743243243243243, 0.0], 
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0],
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0]]
    
    if score is np.nan or status in ['NEW', 'ERROR']:
        return 0
    elif score > 10 and productType == 'CASH_LOAN':
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)
    elif score > 10 and productType == 'CREDIT_CARD':
        return round(ARlist[int(round(score, -1)/10)-1][1]*check[1]*koef, 2)
    elif score > 10 and productType == 'LOAN':
#         print(score, productType, status)
        return round(ARlist[int(round(score, -1)/10)-1][2]*check[2]*koef, 2)
    elif score > 10 and productType == 'REFINANCE':
        return round(ARlist[int(round(score, -1)/10)-1][3]*check[3]*koef, 2)
    else:
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)

dr['createdAt'] = pd.to_datetime(dr['createdAt'])
dr['payout'] = dr['payout'].astype(float)
dr['payout'] = dr.apply(lambda row: 3800 if row['partner']=='Альфа-Банк' 
                        else row['payout'], axis=1)
dr['payout'] = dr.apply(lambda row: 2550 if row['partner']=='Kredito24' 
                        else row['payout'], axis=1)
dr['productType'] = dr['product'].apply(lambda x: x['productType'])


dr['source'] = dr.apply(lambda row: 'Металинвест Cash' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)

dr = dr[dr['status_x']!='NEW']
dr['score'] = dr['score'].apply(lambda x: x+75 if x==0 else x)
dr['whence']=dr.apply(lambda row: row['whence_y'] if pd.notnull(row['whence_y']) else row['whence_x'],axis=1)
dr['source']=dr.apply(lambda row: row['utm_source_y'] if pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['medium']=dr.apply(lambda row: row['utm_medium_y'] if pd.notnull(row['utm_medium_y']) else row['whence'],axis=1)

sms = dr.groupby('source').agg({'_id':'count'}).to_excel('sms.xlsx')

dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
                      pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
                      pd.notnull(row['sourceGlobal']) else row['source'],axis=1)
dr['source']=dr.apply(lambda row: 'Город Тройка' if 
                      row['utm_content']=='gorodtroika' else row['source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub'  else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term','utm_content', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

odin = dr[dr['dt_reg'].notnull()]
odin=odin[odin['product_type']=='LOAN']
odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)

278211
567548; 557548; 547548; 537548; 527548; 517548; 507548; 497548; 487548; 477548; 467548; 457548; 447548; 437548; 427548; 417548; 407548; 397548; 387548; 377548; 367548; 357548; 347548; 337548; 327548; 317548; 307548; 297548; 287548; 277548; 267548; 257548; 247548; 237548; 227548; 217548; 207548; 197548; 187548; 177548; 167548; 157548; 147548; 137548; 127548; 117548; 107548; 97548; 87548; 77548; 67548; 57548; 47548; 37548; 27548; 17548; 7548; 93931; 83931; 73931; 63931; 53931; 43931; 33931; 23931; 13931; 3931; 55997; 45997; 35997; 25997; 15997; 5997; 55997 raz
34863; 33863; 32863; 31863; 30863; 29863; 28863; 27863; 26863; 25863; 24863; 23863; 22863; 21863; 20863; 19863; 18863; 17863; 16863; 15863; 14863; 13863; 12863; 11863; 10863; 9863; 8863; 7863; 6863; 5863; 4863; 3863; 2863; 1863; 863; 

In [11]:
dr['dt'] = pd.to_datetime(dr['dt'])
dr['month'] = dr['dt'].apply(lambda x: datetime.strftime(x, '%Y-%m-%B'))


In [12]:
#regs = get_kibana_reg(es, since,till)

In [13]:
#regs

In [14]:
# regs = regs[regs['utm_source'].isin(['admitad','leadgid2','advcake','leadgid'])]
# regs['week'] = (pd.to_datetime(regs['dt_reg']).dt.week)+1
# # regs_group = regs.groupby(['week','userId','utm_source'],as_index=False).agg({'dt_reg':'count'})
# regs_group = regs.groupby(['week','utm_campaign','utm_source'],as_index=False).agg({'dt_reg':'count'})

# regs_group

In [15]:
dr_noname = dr[dr['source_x'].isin(['Admitad','Adv.Cake','Leadgid','Rubid','Cpahub'])]

In [16]:
dr_noname.head(1)

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,requestId,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_content_x,utm_source_y,utm_term_y,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence,medium,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,utm_content_y,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,month
4,5ff2642aba6ffa0001415898,LINK,2021-01-04 00:41:14.160,2021-01-04 00:41:14.160,REJECTED,"{'_id': None, 'userId': '5ff262389bad3e0001d768e7', 'fullName': 'Абрамунин Денис Витальевич', 'email': 'abrekgorbatyj@gmail.com', 'phone': '79959825755'}",Vivus,"{'_id': 5fe0587131fcc00001239248, 'name': 'Деньги на банковскую карту от Vivus', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}","{'LINK': None, 'NOTE': '140'}",showcase,NaN,345c497f-fe5d-4e66-a0a3-3ba01cbaaad4,5ff2642aba6ffa0001415898,2021-01-04,5fe0587131fcc00001239248,CREDIT_CARD,Деньги на банковскую карту от Vivus,5ff262389bad3e0001d768e7,79959825755,NaN,NaN,NaN,NaN,NaN,NaN,Admitad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT_CARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,5ff262389bad3e0001d768e7,552360,cpa,NaN,admitad,1591470,GA1.2.1880884801.1609720380,1609720380381140309,NaN,23,showcase,cpa,None,False,"{""id"":""071a6a42-f591-4561-b00f-86de2b45ab7e"",""eventType"":""REGISTRATION"",""timestamp"":""2021-01-04T00:34:00.386Z"",""source"":""public"",""indexName"":null,""params"":{""sessionId"":""c293ed1d-a527-4618-b59d-6f22635694cc"",""currentPage"":""/podbor-kreditov-vidacha"",""nextPage"":""/podbor-kreditov-vidacha"",""userId"":""5ff262389bad3e0001d768e7"",""utm_source"":""admitad"",""utm_medium"":""cpa"",""utm_campaign"":""552360"",""transaction_id"":""ea39fc657d8cdd2596ca31e6a3e28f1b"",""productType"":""card"",""utm_term"":""1591470"",""ipAddress"":""94.25.168.43""},""fallbackResponse"":false}",071a6a42-f591-4561-b00f-86de2b45ab7e,REGISTRATION,public,"{'transaction_id': 'ea39fc657d8cdd2596ca31e6a3e28f1b', 'utm_term': '1591470', 'utm_campaign': '552360', 'nextPage': '/podbor-kreditov-vidacha', 'utm_medium': 'cpa', 'ipAddress': '94.25.168.43', 'sessionId': 'c293ed1d-a527-4618-b59d-6f22635694cc', 'currentPage': '/podbor-kreditov-vidacha', 'userId': '5ff262389bad3e0001d768e7', 'productType': 'card', 'utm_source': 'admitad'}",2021-01-04T00:34:00.386Z,NaN,552360,cpa,admitad,1591470,NaN,card,NaN,/podbor-kreditov-vidacha,NaN,2021-01-04 00:34:00,2021-01-04,/podb,/podbor-k,2021-01-January


In [17]:
admitad = dr_noname[dr_noname['source_x']=='Admitad']
admitad['week'] = (pd.to_datetime(admitad['dt_reg']).dt.week)+1
admitad_group = admitad.groupby(['week','utm_campaign_y',
                                 'utm_source_y'
                                 ],as_index=False).agg({'productType_x':'count'})
admitad_group.columns =['week','utm_campaign','utm_source','Заявка']
# #admitad_group['Уникальная заявка']=1
# admitad_group = regs_group.merge(admitad_group,how='left',on=['week','utm_campaign','utm_source'])
# admitad_group= admitad_group[admitad_group['utm_source']=='admitad']
# # admitad_group = admitad_group.groupby(['week','utm_source','utm_term'],
# #                                       as_index=False).agg({'dt_reg':'sum',
# #                                                            'Заявка':'sum',
# #                                                            'Уникальная заявка':'sum'})
admitad_group

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,week,utm_campaign,utm_source,Заявка
0,2.0,1137902,admitad,1
1,2.0,1325011,admitad,1
2,2.0,31434,admitad,2
3,2.0,449953,admitad,9
4,2.0,552360,admitad,2
5,2.0,58534,admitad,2
6,2.0,654535,admitad,2
7,2.0,772336,admitad,1
8,2.0,90289,admitad,4
9,3.0,1325011,admitad,3


In [18]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'UTMContent':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_campaign','utm_source'],as_index=False).agg({'visits':'sum'})


df_admitad = df.merge(admitad_group,how='left',on=['week','utm_campaign','utm_source',])



df_admitad = df_admitad.fillna(0)
#df_admitad['cr_в_регу']= ((df_admitad['dt_reg']/df_admitad['visits'])*100).round(2)
df_admitad['cr_трафик_заявка']= ((df_admitad['Заявка']/df_admitad['visits'])*100).round(2)
#df_admitad['cr_трафик_у_заявка']= ((df_admitad['Уникальная заявка']/df_admitad['visits'])*100).round(2)
#df_admitad['cr_рега_в_у_заявка']= ((df_admitad['Уникальная заявка']/df_admitad['dt_reg'])*100).round(2)
df_admitad = df_admitad.fillna(0)
df_admitad.to_excel(f'reports/admitadCPA.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AQ0a8-baNB6cFzs1_OFmIWAsVwEhjh_ZKlCj3sRvMNQ')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel(f'reports/admitadCPA.xlsx'
                  )
set_with_dataframe(worksheet, df)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [19]:
df_admitad

,week,utm_campaign,utm_source,visits,Заявка,cr_трафик_заявка
0,2,1016220,admitad,1.0,0.0,0.00
1,2,1049044,admitad,1.0,0.0,0.00
2,2,1061986,admitad,1.0,0.0,0.00
3,2,1078326,admitad,3.0,0.0,0.00
4,2,1088335,admitad,1.0,0.0,0.00
...,...,...,...,...,...,...
2225,51,1325011,admitad,1.0,0.0,0.00
2226,51,1330093,admitad,10.0,0.0,0.00
2227,51,1357296,admitad,1.0,0.0,0.00
2228,51,1512587,admitad,6.0,0.0,0.00


In [20]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_content':result[i]["dimensions"][3]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            #'UTMContent':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_source','utm_campaign','utm_content'],as_index=False).agg({'visits':'sum'})

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [21]:
df.tail(200)

,week,utm_source,utm_campaign,utm_content,visits
3608,49,advcake,leadssu,169364,1.0
3609,49,advcake,leadssu,169722,3.0
3610,49,advcake,leadssu,19464,1.0
3611,49,advcake,leadssu,52356,54.0
3612,49,advcake,leadssu,654,4.0
3613,49,advcake,leadssu,75872,1.0
3614,49,advcake,leadssu,9899,4.0
3615,49,advcake,leadssu,{preview_mode:affiliate_id},1.0
3616,49,advcake,mastertarget,Smarts,10.0
3617,49,advcake,mastertarget,Webmaster9876543210,1.0


In [22]:
# regs = regs[regs['utm_source'].isin(['admitad','leadgid2','advcake','leadgid'])]
# regs['week'] = (pd.to_datetime(regs['dt_reg']).dt.week)+1
# # regs_group = regs.groupby(['week','userId','utm_source'],as_index=False).agg({'dt_reg':'count'})
# regs_group = regs.groupby(['week','utm_campaign','utm_content','utm_source'],as_index=False).agg({'dt_reg':'count'})

# regs_group

In [23]:
advcake = dr_noname[dr_noname['source_x']=='Adv.Cake']
advcake['week'] = (pd.to_datetime(advcake['dt_reg']).dt.week)+1
advcake_group = advcake.groupby(['week',
                                 'utm_source_y','utm_campaign_y',
                                 'utm_content_y'],as_index=False).agg({'productType_x':'count'})
advcake_group.columns =['week','utm_source','utm_campaign','utm_content','Заявка']
#advcake_group['Уникальная заявка']=1
# advcake_group = regs_group[regs_group['utm_source']=='advcake'].merge(advcake_group,how='left',on=['week'
#                                                                                                    ,'utm_campaign',
#                                                                                        'utm_content',
#                                                                                       'utm_source'])
advcake_group = advcake_group.groupby(['week','utm_source','utm_campaign','utm_content'],
                                      as_index=False).agg({
                                                           'Заявка':'sum'
                                                           })
df_advacake = df.merge(advcake_group,how='left',on=['week','utm_source','utm_campaign','utm_content'])

df_advacake = df_advacake.fillna(0)
#df_advacake['cr_в_регу']= ((df_advacake['dt_reg']/df_advacake['visits'])*100).round(2)
df_advacake['cr_трафик_заявка']= ((df_advacake['Заявка']/df_advacake['visits'])*100).round(2)
#df_advacake['cr_трафик_у_заявка']= ((df_advacake['Уникальная заявка']/df_advacake['visits'])*100).round(2)
#df_advacake['cr_рега_в_у_заявка']= ((df_advacake['Уникальная заявка']/df_advacake['dt_reg'])*100).round(2)
df_advacake = df_advacake.fillna(0)
df_advacake.to_excel(f'reports/df_advacakeCPA.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AQ0a8-baNB6cFzs1_OFmIWAsVwEhjh_ZKlCj3sRvMNQ')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel(f'reports/df_advacakeCPA.xlsx'
                  )
set_with_dataframe(worksheet, df)
df_advacake

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,week,utm_source,utm_campaign,utm_content,visits,Заявка,cr_трафик_заявка
0,2,advcake,affiliate,gorodtroika,147.0,0.0,0.00
1,2,advcake,affiliate,workle,98.0,12.0,12.24
2,2,advcake,cityads,18CN,1.0,0.0,0.00
3,2,advcake,cityads,1HLZ,1.0,0.0,0.00
4,2,advcake,cityads,1jA8m3,4.0,0.0,0.00
...,...,...,...,...,...,...,...
3803,51,advcake,saleads,55b742e0-18af-11ec-9c01-a1b70db0b6d8,2.0,0.0,0.00
3804,51,advcake,saleads,626e9a10-fce0-11eb-a54b-1d9d5fe93b32,3.0,0.0,0.00
3805,51,advcake,unicom24,180390,21.0,0.0,0.00
3806,51,advcake,unicom24,180394,3.0,0.0,0.00


In [24]:
advcake_group

,week,utm_source,utm_campaign,utm_content,Заявка
0,2.0,advcake,affiliate,workle,12
1,2.0,advcake,cityads,WnJMoQ,1
2,2.0,advcake,guruleads,3375,4
3,2.0,advcake,leadssu,142831,2
4,2.0,advcake,leadssu,150307,1
5,2.0,advcake,leadssu,151291,2
6,2.0,advcake,leadssu,52356,2
7,2.0,advcake,mastertarget,gogogo,4
8,2.0,advcake,mastertarget,kreditagregator,12
9,2.0,advcake,mastertarget,maximedia,1


In [25]:
df

,Unnamed: 0,week,utm_source,utm_campaign,utm_content,visits,Заявка,cr_трафик_заявка
0,0,2,advcake,affiliate,gorodtroika,147,0,0.00
1,1,2,advcake,affiliate,workle,98,12,12.24
2,2,2,advcake,cityads,18CN,1,0,0.00
3,3,2,advcake,cityads,1HLZ,1,0,0.00
4,4,2,advcake,cityads,1jA8m3,4,0,0.00
...,...,...,...,...,...,...,...,...
3803,3803,51,advcake,saleads,55b742e0-18af-11ec-9c01-a1b70db0b6d8,2,0,0.00
3804,3804,51,advcake,saleads,626e9a10-fce0-11eb-a54b-1d9d5fe93b32,3,0,0.00
3805,3805,51,advcake,unicom24,180390,21,0,0.00
3806,3806,51,advcake,unicom24,180394,3,0,0.00


In [26]:
Leadgid = dr_noname[dr_noname['source_x']=='Leadgid']
#Leadgid['utm_content'].unique()

Leadgid['week'] = (pd.to_datetime(Leadgid['dt_reg']).dt.week)+1
Leadgid_group = Leadgid.groupby(['week',
                                 'utm_source_y',
                                 'utm_campaign_y'],as_index=False).agg({'productType_x':'count'})
Leadgid_group.columns =['week','utm_source','utm_campaign','Заявка']
#Leadgid_group['Уникальная заявка']=1
Leadgid_group

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,week,utm_source,utm_campaign,Заявка
0,2.0,leadgid,25301,3
1,2.0,leadgid,58668,2
2,2.0,leadgid,65164,1
3,3.0,leadgid,62743,2
4,4.0,leadgid,26649,2
5,4.0,leadgid2,49758,4
6,5.0,leadgid,58668,1
7,5.0,leadgid,65164,1
8,7.0,leadgid2,49758,1
9,8.0,leadgid,25521,3


In [27]:
Leadgid

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,requestId,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_content_x,utm_source_y,utm_term_y,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence,medium,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,utm_content_y,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,month,week
327,5ff71bfe7976dd000132bac9,LINK,2021-01-07 14:34:38.397,2021-01-07 14:34:38.397,APPROVED,"{'_id': None, 'userId': '5ff71b8a9bad3e0001d8c78a', 'fullName': 'Беломестнов Дмитрий Андреевич', 'email': 'dmbelomestnov26@gmail.com', 'phone': '79230026268'}",Экспобанк,"{'_id': 5f509227b50341000118d7c2, 'name': 'Лёгкий кредит', 'productType': 'CASH_LOAN', 'productName': 'Кредит наличными'}","{'LINK': None, 'NOTE': '5160'}",showcase,LEADGID_PARTNER,bb9f0a08-d676-4613-a1ac-daacd7cac7f6,5ff71bfe7976dd000132bac9,2021-01-07,5f509227b50341000118d7c2,CASH_LOAN,Лёгкий кредит,5ff71b8a9bad3e0001d8c78a,79230026268,NaN,NaN,False,"{""id"":""f82eb406-3921-4aa8-af98-392d9554c325"",""eventType"":""POSTBACK"",""timestamp"":""2021-02-20T06:36:37.088Z"",""source"":""private"",""indexName"":null,""params"":{""partnerName"":""leadgid"",""transitionId"":""5ff71bfe7976dd000132bac9"",""payout"":""140"",""status"":""APPROVED""},""fallbackResponse"":false}",f82eb406-3921-4aa8-af98-392d9554c325,POSTBACK,Leadgid,"{'payout': '140', 'partnerName': 'leadgid', 'transitionId': '5ff71bfe7976dd000132bac9', 'status': 'APPROVED'}",2021-02-20T06:36:37.088Z,NaN,NaN,NaN,NaN,NaN,NaN,CASH_LOAN,NaN,NaN,NaN,5ff71bfe7976dd000132bac9,leadgid,140.0,APPROVED,NaN,NaN,NaN,NaN,2021-02-20 06:36:37,2021-02-20,5ff71b8a9bad3e0001d8c78a,58668,cpa,NaN,leadgid,NaN,GA1.2.205386755.1610029969,1605414916651765824,NaN,1,showcase,cpa,None,False,"{""id"":""4095a323-a67b-432b-b682-fc2799a3558b"",""eventType"":""REGISTRATION"",""timestamp"":""2021-01-07T14:33:21.066Z"",""source"":""public"",""indexName"":null,""params"":{""sessionId"":""61b9ead4-ff4a-4bfd-aa07-b4afbc82a111"",""yandexClientID"":""1605414916651765824"",""currentPage"":""/podbor-kreditov-vidacha"",""nextPage"":""/podbor-kreditov-vidacha"",""userId"":""5ff71b8a9bad3e0001d8c78a"",""utm_source"":""leadgid"",""utm_medium"":""cpa"",""utm_campaign"":""58668"",""transaction_id"":""10242bfcc67ebb342b8c2203fb5a38"",""productType"":""cash"",""ipAddress"":""91.105.184.36""},""fallbackResponse"":false}",4095a323-a67b-432b-b682-fc2799a3558b,REGISTRATION,public,"{'transaction_id': '10242bfcc67ebb342b8c2203fb5a38', 'utm_campaign': '58668', 'yandexClientID': '1605414916651765824', 'nextPage': '/podbor-kreditov-vidacha', 'utm_medium': 'cpa', 'ipAddress': '91.105.184.36', 'sessionId': '61b9ead4-ff4a-4bfd-aa07-b4afbc82a111', 'currentPage': '/podbor-kreditov-vidacha', 'userId': '5ff71b8a9bad3e0001d8c78a', 'productType': 'cash', 'utm_source': 'leadgid'}",2021-01-07T14:33:21.066Z,NaN,58668,cpa,leadgid,NaN,NaN,cash,1605414916651765824,/podbor-kreditov-vidacha,NaN,2021-01-07 14:33:21,2021-01-07,/podb,/podbor-k,2021-01-January,2.0
432,5ff8718c53668d0001f5ca10,LINK,2021-01-08 14:51:56.031,2021-01-08 14:51:56.031,REJECTED,"{'_id': None, 'userId': '5ff86d049bad3e0001d92bac', 'fullName': 'Астахова Лариса Юрьевна', 'email': '89067309835@yandex.ru', 'phone': '79067309835'}",Альфа-Банк,"{'_id': 5ab391b1ed55ff0009a3ce5f, 'name': 'Кредит наличными', 'productType': 'CASH_LOAN', 'productName': 'Кредит наличными'}","{'LINK': None, 'NOTE': '926'}",showcase,,4b438031-7349-4a65-bb31-a33402086e01,5ff8718c53668d0001f5ca10,202

In [28]:
# regs_l = regs[regs['utm_source'].isin(['leadgid2','leadgid'])]
# regs_l['week'] = (pd.to_datetime(regs_l['dt_reg']).dt.week)+1
# # regs_group = regs.groupby(['week','userId','utm_source'],as_index=False).agg({'dt_reg':'count'})
# regs_group = regs_l.groupby(['week','utm_campaign','utm_source'],as_index=False).agg({'dt_reg':'count'})

# #regs_group

In [29]:
# Leadgid_group1 = regs_group[regs_group['utm_source'].isin(['leadgid2',
#                                                           'leadgid'])].merge(Leadgid_group
#                                                                              ,how='left',on=['week',
#                                                                                 'utm_source','utm_campaign'])
Leadgid_group1 = Leadgid_group

In [30]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign",
    date1 ="2021-01-04",
    date2 = "yesterday",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=@'lead'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'UTMContent':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_source','utm_campaign'],as_index=False).agg({'visits':'sum'})

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [31]:
Leadgid_group2 = Leadgid_group1.groupby(['week','utm_source','utm_campaign'],
                                      as_index=False).agg({
                                                           'Заявка':'sum',
                                                           })
Leadgid_group3 = df.merge(Leadgid_group2,how='left',on=['week','utm_source','utm_campaign'])

Leadgid_group3 = Leadgid_group3.fillna(0)
#Leadgid_group3['cr_в_регу']= ((Leadgid_group3['dt_reg']/Leadgid_group3['visits'])*100).round(2)
Leadgid_group3['cr_трафик_заявка']= ((Leadgid_group3['Заявка']/Leadgid_group3['visits'])*100).round(2)
#Leadgid_group['cr_трафик_у_заявка']= ((Leadgid_group['Уникальная заявка']/Leadgid_group['visits'])*100).round(2)
#Leadgid_group['cr_рега_в_у_заявка']= ((Leadgid_group['Уникальная заявка']/Leadgid_group['dt_reg'])*100).round(2)
Leadgid_group3 = Leadgid_group3.fillna(0)
Leadgid_group3.to_excel(f'reports/Leadgid_group.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AQ0a8-baNB6cFzs1_OFmIWAsVwEhjh_ZKlCj3sRvMNQ')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel(f'reports/Leadgid_group.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [32]:
Leadgid_group3

,week,utm_source,utm_campaign,visits,Заявка,cr_трафик_заявка
0,2,leadgid,25301,1.0,3.0,300.00
1,2,leadgid,42009,18.0,0.0,0.00
2,2,leadgid,55700,1.0,0.0,0.00
3,2,leadgid,56884,1.0,0.0,0.00
4,2,leadgid,58668,18.0,2.0,11.11
5,2,leadgid,65164,28.0,1.0,3.57
6,2,leadgid3,25265,1.0,0.0,0.00
7,2,leadgid3,6874,1.0,0.0,0.00
8,2,leads2,126747,2.0,0.0,0.00
9,3,leadgid,14088,1.0,0.0,0.00


In [33]:
cpahub = dr_noname[dr_noname['source_x']=='Cpahub']
#cpahub
# #Leadgid['utm_content'].unique()

# Leadgid['week'] = (pd.to_datetime(Leadgid['dt_reg']).dt.week)+1
# Leadgid_group = Leadgid.groupby(['week',
#                                  'utm_source_y',
#                                  'utm_campaign_y'],as_index=False).agg({'productType_x':'count'})
# Leadgid_group.columns =['week','utm_source','utm_campaign','Заявка']
# #Leadgid_group['Уникальная заявка']=1
# Leadgid_group

In [34]:
cpahub['web'] = cpahub['whence_x'].str[38:]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
cpahub['web']=cpahub.web.str.strip('_')
cpahub.head(1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,requestId,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_content_x,utm_source_y,utm_term_y,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence,medium,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,utm_content_y,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,month,web
629,5ffae24be08dfe0001e18c0c,LINK,2021-01-10 11:17:31.648,2021-01-10 11:17:31.648,REJECTED,"{'_id': None, 'userId': '5ffadda99bad3e0001d9dd74'}",Vivus,"{'_id': 5abb6f9a21f7ea00074349d8, 'name': 'Vivus', 'productType': 'LOAN', 'productName': 'Займ'}","{'LINK': None, 'NOTE': '140'}",cpahub2_1020b4221ec5b2b7cf8716f63f8771_13806,,NaN,5ffae24be08dfe0001e18c0c,2021-01-10,5abb6f9a21f7ea00074349d8,LOAN,Vivus,5ffadda99bad3e0001d9dd74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cpahub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,cpahub2_1020b4221ec5b2b7cf8716f63f8771_13806,cpahub2_1020b4221ec5b2b7cf8716f63f8771_13806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2021-01-January,13806


In [36]:
cpahub.groupby(['web','partner','status_x']).agg({'userId_x':'count','payout':'sum'})

userId_x   payout
web   partner           status_x                   
      Восточный Банк    APPROVED         1    150.0
      МТС Банк          REJECTED         1      0.0
13058 LIME              REJECTED         1      0.0
      Max.Credit        REJECTED         1      0.0
13806  Vivus            REJECTED         1      0.0
      LIME              REJECTED         1      0.0
      Веб-Займ          APPROVED         2    235.0
      Турбозайм         REJECTED         1      0.0
14049 CreditPlus        APPROVED         1    300.0
14957 Е заем            ACCEPTED         1      0.0
15884  Vivus            APPROVED         1   3150.0
      FastMoney         ACCEPTED         6      0.0
                        APPROVED         1   1350.0
      LIME              APPROVED         4  11020.0
                        REJECTED         4      0.0
      Max.Credit        REJECTED         2      0.0
      MoneyMan          ACCEPTED         6      0.0
                        APPROVED         6  24000.0
      Moneza            ACCEPTED         1      0.0
                        APPROVED         1   1125.0
      Веб-Займ          APPROVED         2   3285.0
                        REJECTED         1      0.0
      Деньги Сразу      APPROVED         1    780.0
      До Зарплаты       APPROVED         1   2100.0
                        REJECTED         1      0.0
      Е заем            ACCEPTED         3      0.0
      Займер            APPROVED         2   1600.0
      МигКредит         ACCEPTED         1      0.0
      Отличные наличные ACCEPTED         1      0.0
                        APPROVED         4   6200.0
                        REJECTED         5      0.0
      Срочноденьги      APPROVED         3    891.0
      Турбозайм         REJECTED         1      0.0
      еКапуста          APPROVED         2   4650.0
                        REJECTED         1      0.0
366    Vivus            REJECTED         1      0.0
      МигКредит         ACCEPTED         1      0.0
42    еКапуста          REJECTED         1      0.0
9286  FastMoney         ACCEPTED         1      0.0
      MoneyMan          ACCEPTED         1      0.0
      Веб-Займ          APPROVED         1   1700.0

In [37]:
cpahub['week'] = (pd.to_datetime(cpahub['dt']).dt.week)+1
cpahub_group = cpahub.groupby(['week',
                                 'web'],as_index=False).agg({'productType_x':'count'})
cpahub_group.columns =['week','utm_campaign','Заявка']
#Leadgid_group['Уникальная заявка']=1
cpahub_group

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,week,utm_campaign,Заявка
0,2,13806,2
1,3,366,1
2,5,13806,1
3,6,13806,1
4,8,13806,1
5,9,,2
6,10,42,1
7,15,9286,1
8,16,13058,1
9,18,14049,1


In [38]:
#regs = get_kibana_reg(es, since,till)

In [39]:
#regs['utm_campaign'].value_counts()

In [40]:
# regsc = regs[regs['utm_source'].isin(['cpahub2','cpahub'])]
# regsc['week'] = (pd.to_datetime(regsc['dt_reg']).dt.week)+1
# # regs_group = regs.groupby(['week','userId','utm_source'],as_index=False).agg({'dt_reg':'count'})
# regs_group = regsc.groupby(['week','utm_campaign','utm_source'],as_index=False).agg({'dt_reg':'count'})

# regsc

In [41]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=@'cpahub'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'UTMContent':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_source','utm_campaign'],as_index=False).agg({'visits':'sum'})

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [42]:
cpahub_group1 = df.merge(cpahub_group,how='left',on=['week','utm_campaign'])

In [43]:
cpahub_group

,week,utm_campaign,Заявка
0,2,13806,2
1,3,366,1
2,5,13806,1
3,6,13806,1
4,8,13806,1
5,9,,2
6,10,42,1
7,15,9286,1
8,16,13058,1
9,18,14049,1


In [44]:
cpahub_group1

,week,utm_source,utm_campaign,visits,Заявка
0,2,cpahub,2849,1.0,NaN
1,2,cpahub,42,36.0,NaN
2,2,cpahub,7726,2.0,NaN
3,2,cpahub,9286,8.0,NaN
4,2,cpahub2,13806,2.0,2.0
5,3,cpahub,366,5.0,1.0
6,3,cpahub,42,23.0,NaN
7,3,cpahub,7726,2.0,NaN
8,3,cpahub,9286,7.0,NaN
9,4,cpahub,14554,41.0,NaN


In [45]:
cpahub_group1 = cpahub_group1.fillna(0)
#cpahub_group1['cr_в_регу']= ((cpahub_group1['dt_reg']/cpahub_group1['visits'])*100).round(2)
cpahub_group1['cr_трафик_заявка']= ((cpahub_group1['Заявка']/cpahub_group1['visits'])*100).round(2)
#Leadgid_group['cr_трафик_у_заявка']= ((Leadgid_group['Уникальная заявка']/Leadgid_group['visits'])*100).round(2)
#Leadgid_group['cr_рега_в_у_заявка']= ((Leadgid_group['Уникальная заявка']/Leadgid_group['dt_reg'])*100).round(2)
cpahub_group1 = cpahub_group1.fillna(0)
cpahub_group1.to_excel(f'reports/cpahub_group1.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AQ0a8-baNB6cFzs1_OFmIWAsVwEhjh_ZKlCj3sRvMNQ')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel(f'reports/cpahub_group1.xlsx'
                  )
set_with_dataframe(worksheet, df)